In [19]:
'''
Author: ChenHJ
Date: 2022-10-30 18:47:32
LastEditors: ChenHJ
LastEditTime: 2022-10-30 18:48:24
FilePath: /chenhj/0302code/obs_divide_SASMR.ipynb
Aim: 
10/30/2022: 
这个脚本用于计算在观测-再分析数据中, 印度半岛北部和南部降水的不同的相关环流, 海温过程等等
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import Cdo
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof

## 计算印度半岛北部、南部降水相关的环流、海温等等

### 读取数据

In [20]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)

preAIR = xr.open_dataarray("/home/ys17-23/Extension/All_India_Rainfall_index/AIR_mmperday.nc")
preAIR_JJA = ca.p_time(preAIR, 6, 8, True)
preAIR_JJA = preAIR_JJA.sel(time=(preAIR_JJA.time.dt.year>=1979) & (preAIR_JJA.time.dt.year <=2014))


fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
uERA5_ver_JJA = ca.p_time(uERA5, 6, 8, True)
vERA5_ver_JJA = ca.p_time(vERA5, 6, 8, True)
qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True)
spERA5_JJA = ca.p_time(spERA5, 6, 8, True)
wERA5_JJA = ca.p_time(wERA5, 6, 8, True)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)

ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)

divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))


#   calculate the waver vapor vertical intergration
ptop = 100 * 100
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= 100) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
uq_sl_ERA5_JJA = uERA5_ver_JJA * qERA5_ver_JJA.data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA * qERA5_ver_JJA.data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg.data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg.data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "[m/s][g/kg]"
vqERA5_ver_JJA.attrs["units"] = "[m/s][g/kg]"

# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)

### 对降水进行分区

In [21]:
# 北部
lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]
nIndia_N = 32.5
nIndia_S = 25.0
nIndia_W = 70.0
nIndia_E = 85.0
lat_nIndia_range = lat[(lat >= nIndia_S) & (lat <= nIndia_N)]
lon_nIndia_range = lon[(lon >= nIndia_W) & (lon <= nIndia_E)]

preGPCP_nIndia_JJA = ca.cal_lat_weighted_mean(preGPCP_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range)).mean(dim="lon", skipna=True)

# standardize
preGPCP_nIndia_JJA_nor = ca.standardize(preGPCP_nIndia_JJA)

# 南部
sIndia_N = 25.0
sIndia_S = 7.5
sIndia_W = 70.0
sIndia_E = 85.0
lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

preGPCP_sIndia_JJA = ca.cal_lat_weighted_mean(preGPCP_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range)).mean(dim="lon", skipna=True)

# standardize
preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA)

# 全印度
India_N = 32.5
India_S = 7.5
India_W = 70.0
India_E = 85.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]

preGPCP_India_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range))

# standardize
preGPCP_India_JJA_nor = ca.standardize(preGPCP_India_JJA)

### 计算降水场回归、环流场回归

In [22]:
# 降水场
(
    nIndR_GPCP_pre_slope,
    nIndR_GPCP_pre_intercept,
    nIndR_GPCP_pre_rvalue,
    nIndR_GPCP_pre_pvalue,
    nIndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, preGPCP_JJA)

(
    sIndR_GPCP_pre_slope,
    sIndR_GPCP_pre_intercept,
    sIndR_GPCP_pre_rvalue,
    sIndR_GPCP_pre_pvalue,
    sIndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, preGPCP_JJA)

(
    IndR_GPCP_pre_slope,
    IndR_GPCP_pre_intercept,
    IndR_GPCP_pre_rvalue,
    IndR_GPCP_pre_pvalue,
    IndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, preGPCP_JJA)

In [23]:
# 环流场/风场
preGPCP_nIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time
preGPCP_sIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time
(
    nIndR_ERA5_hgt_slope,
    nIndR_ERA5_hgt_intercept,
    nIndR_ERA5_hgt_rvalue,
    nIndR_ERA5_hgt_pvalue,
    nIndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_ERA5_u_slope,
    nIndR_ERA5_u_intercept,
    nIndR_ERA5_u_rvalue,
    nIndR_ERA5_u_pvalue,
    nIndR_ERA5_u_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_ERA5_v_slope,
    nIndR_ERA5_v_intercept,
    nIndR_ERA5_v_rvalue,
    nIndR_ERA5_v_pvalue,
    nIndR_ERA5_v_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ERA5_hgt_slope,
    sIndR_ERA5_hgt_intercept,
    sIndR_ERA5_hgt_rvalue,
    sIndR_ERA5_hgt_pvalue,
    sIndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ERA5_u_slope,
    sIndR_ERA5_u_intercept,
    sIndR_ERA5_u_rvalue,
    sIndR_ERA5_u_pvalue,
    sIndR_ERA5_u_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ERA5_v_slope,
    sIndR_ERA5_v_intercept,
    sIndR_ERA5_v_rvalue,
    sIndR_ERA5_v_pvalue,
    sIndR_ERA5_v_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ERA5_hgt_slope,
    IndR_ERA5_hgt_intercept,
    IndR_ERA5_hgt_rvalue,
    IndR_ERA5_hgt_pvalue,
    IndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ERA5_u_slope,
    IndR_ERA5_u_intercept,
    IndR_ERA5_u_rvalue,
    IndR_ERA5_u_pvalue,
    IndR_ERA5_u_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ERA5_v_slope,
    IndR_ERA5_v_intercept,
    IndR_ERA5_v_rvalue,
    IndR_ERA5_v_pvalue,
    IndR_ERA5_v_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

In [24]:
# 垂直运动场和散度场
(
    nIndR_ERA5_w_slope,
    nIndR_ERA5_w_intercept,
    nIndR_ERA5_w_rvalue,
    nIndR_ERA5_w_pvalue,
    nIndR_ERA5_w_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, wERA5_JJA.sel(level=500.0))

(
    nIndR_ERA5_div_slope,
    nIndR_ERA5_div_intercept,
    nIndR_ERA5_div_rvalue,
    nIndR_ERA5_div_pvalue,
    nIndR_ERA5_div_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, divERA5_ver_JJA.sel(level=[200.0, 850.0]))

(
    sIndR_ERA5_w_slope,
    sIndR_ERA5_w_intercept,
    sIndR_ERA5_w_rvalue,
    sIndR_ERA5_w_pvalue,
    sIndR_ERA5_w_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, wERA5_JJA.sel(level=500.0))

(
    sIndR_ERA5_div_slope,
    sIndR_ERA5_div_intercept,
    sIndR_ERA5_div_rvalue,
    sIndR_ERA5_div_pvalue,
    sIndR_ERA5_div_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, divERA5_ver_JJA.sel(level=[200.0, 850.0]))

(
    IndR_ERA5_w_slope,
    IndR_ERA5_w_intercept,
    IndR_ERA5_w_rvalue,
    IndR_ERA5_w_pvalue,
    IndR_ERA5_w_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, wERA5_JJA.sel(level=500.0))

(
    IndR_ERA5_div_slope,
    IndR_ERA5_div_intercept,
    IndR_ERA5_div_rvalue,
    IndR_ERA5_div_pvalue,
    IndR_ERA5_div_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, divERA5_ver_JJA.sel(level=[200.0, 850.0]))

In [25]:
level = nIndR_ERA5_hgt_slope.coords["level"]

nIndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], nIndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], nIndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014 "),
)

nIndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_u_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_u_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_u_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_u_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_u_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_v_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_v_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_v_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_v_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_v_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_w_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_w_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_w_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_w_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_w_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_w_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_div_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_div_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_div_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_div_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_div_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_div_hypothesis.data),
    ),
    coords=dict(
        level=nIndR_ERA5_div_slope.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="div fields of ERA5 regress onto 1979-2014 "),
)

sIndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], sIndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], sIndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014 "),
)

sIndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_u_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_u_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_u_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_u_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_u_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_v_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_v_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_v_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_v_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_v_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_w_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_w_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_w_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_w_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_w_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_w_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_div_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_div_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_div_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_div_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_div_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_div_hypothesis.data),
    ),
    coords=dict(
        level=sIndR_ERA5_div_slope.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="div fields of ERA5 regress onto 1979-2014 "),
)

IndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], IndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], IndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], IndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014 "),
)

IndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_u_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_u_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_u_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_u_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_u_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_v_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_v_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_v_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_v_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_v_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_w_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_w_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_w_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_w_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_w_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_w_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_div_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_div_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_div_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_div_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_div_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_div_hypothesis.data),
    ),
    coords=dict(
        level=IndR_ERA5_div_slope.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="div fields of ERA5 regress onto 1979-2014 "),
)

nIndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_GPCP_pre_regress.nc")
nIndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_hgt_regress.nc")
nIndR_ERA5_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_u_regress.nc")
nIndR_ERA5_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_v_regress.nc")
nIndR_ERA5_w_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_w_regress.nc")
nIndR_ERA5_div_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_div_regress.nc")

sIndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_GPCP_pre_regress.nc")
sIndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_hgt_regress.nc")
sIndR_ERA5_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_u_regress.nc")
sIndR_ERA5_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_v_regress.nc")
sIndR_ERA5_w_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_w_regress.nc")
sIndR_ERA5_div_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_div_regress.nc")

IndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_GPCP_pre_regress.nc")
IndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_hgt_regress.nc")
IndR_ERA5_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_u_regress.nc")
IndR_ERA5_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_v_regress.nc")
IndR_ERA5_w_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_w_regress.nc")
IndR_ERA5_div_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_div_regress.nc")

### 计算前期冬季、前期春期、同期夏季、后期秋季、后期冬季的海温回归异常

In [26]:
# 印度半岛北部降水
# 前期冬季
tmp = preGPCP_nIndia_JJA_nor[1:].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    nIndR_ERA5_sstDJF_slope,
    nIndR_ERA5_sstDJF_intercept,
    nIndR_ERA5_sstDJF_rvalue,
    nIndR_ERA5_sstDJF_pvalue,
    nIndR_ERA5_sstDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)

# 前期春季
tmp = preGPCP_nIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_MAM.time
(
    nIndR_ERA5_sstMAM_slope,
    nIndR_ERA5_sstMAM_intercept,
    nIndR_ERA5_sstMAM_rvalue,
    nIndR_ERA5_sstMAM_pvalue,
    nIndR_ERA5_sstMAM_hypothesis,
) = ca.dim_linregress(tmp, sstHad_MAM)

# 同期夏季
tmp = preGPCP_nIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_JJA.time
(
    nIndR_ERA5_sstJJA_slope,
    nIndR_ERA5_sstJJA_intercept,
    nIndR_ERA5_sstJJA_rvalue,
    nIndR_ERA5_sstJJA_pvalue,
    nIndR_ERA5_sstJJA_hypothesis,
) = ca.dim_linregress(tmp, sstHad_JJA)

# 后期秋季
tmp = preGPCP_nIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_SON.time
(
    nIndR_ERA5_sstSON_slope,
    nIndR_ERA5_sstSON_intercept,
    nIndR_ERA5_sstSON_rvalue,
    nIndR_ERA5_sstSON_pvalue,
    nIndR_ERA5_sstSON_hypothesis,
) = ca.dim_linregress(tmp, sstHad_SON)

# 后期冬季
tmp = preGPCP_nIndia_JJA_nor[:-1].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    nIndR_ERA5_sstnDJF_slope,
    nIndR_ERA5_sstnDJF_intercept,
    nIndR_ERA5_sstnDJF_rvalue,
    nIndR_ERA5_sstnDJF_pvalue,
    nIndR_ERA5_sstnDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)

# 印度半岛南部降水
# 前期冬季
tmp = preGPCP_sIndia_JJA_nor[1:].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    sIndR_ERA5_sstDJF_slope,
    sIndR_ERA5_sstDJF_intercept,
    sIndR_ERA5_sstDJF_rvalue,
    sIndR_ERA5_sstDJF_pvalue,
    sIndR_ERA5_sstDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)

# 前期春季
tmp = preGPCP_sIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_MAM.time
(
    sIndR_ERA5_sstMAM_slope,
    sIndR_ERA5_sstMAM_intercept,
    sIndR_ERA5_sstMAM_rvalue,
    sIndR_ERA5_sstMAM_pvalue,
    sIndR_ERA5_sstMAM_hypothesis,
) = ca.dim_linregress(tmp, sstHad_MAM)

# 同期夏季
tmp = preGPCP_sIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_JJA.time
(
    sIndR_ERA5_sstJJA_slope,
    sIndR_ERA5_sstJJA_intercept,
    sIndR_ERA5_sstJJA_rvalue,
    sIndR_ERA5_sstJJA_pvalue,
    sIndR_ERA5_sstJJA_hypothesis,
) = ca.dim_linregress(tmp, sstHad_JJA)

# 后期秋季
tmp = preGPCP_sIndia_JJA_nor.copy()
tmp.coords["time"] = sstHad_SON.time
(
    sIndR_ERA5_sstSON_slope,
    sIndR_ERA5_sstSON_intercept,
    sIndR_ERA5_sstSON_rvalue,
    sIndR_ERA5_sstSON_pvalue,
    sIndR_ERA5_sstSON_hypothesis,
) = ca.dim_linregress(tmp, sstHad_SON)

# 后期冬季
tmp = preGPCP_sIndia_JJA_nor[:-1].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    sIndR_ERA5_sstnDJF_slope,
    sIndR_ERA5_sstnDJF_intercept,
    sIndR_ERA5_sstnDJF_rvalue,
    sIndR_ERA5_sstnDJF_pvalue,
    sIndR_ERA5_sstnDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)

# 全印度半岛降水
# 前期冬季
tmp = preGPCP_India_JJA_nor[1:].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    IndR_ERA5_sstDJF_slope,
    IndR_ERA5_sstDJF_intercept,
    IndR_ERA5_sstDJF_rvalue,
    IndR_ERA5_sstDJF_pvalue,
    IndR_ERA5_sstDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)

# 前期春季
tmp = preGPCP_India_JJA_nor.copy()
tmp.coords["time"] = sstHad_MAM.time
(
    IndR_ERA5_sstMAM_slope,
    IndR_ERA5_sstMAM_intercept,
    IndR_ERA5_sstMAM_rvalue,
    IndR_ERA5_sstMAM_pvalue,
    IndR_ERA5_sstMAM_hypothesis,
) = ca.dim_linregress(tmp, sstHad_MAM)

# 同期夏季
tmp = preGPCP_India_JJA_nor.copy()
tmp.coords["time"] = sstHad_JJA.time
(
    IndR_ERA5_sstJJA_slope,
    IndR_ERA5_sstJJA_intercept,
    IndR_ERA5_sstJJA_rvalue,
    IndR_ERA5_sstJJA_pvalue,
    IndR_ERA5_sstJJA_hypothesis,
) = ca.dim_linregress(tmp, sstHad_JJA)

# 后期秋季
tmp = preGPCP_India_JJA_nor.copy()
tmp.coords["time"] = sstHad_SON.time
(
    IndR_ERA5_sstSON_slope,
    IndR_ERA5_sstSON_intercept,
    IndR_ERA5_sstSON_rvalue,
    IndR_ERA5_sstSON_pvalue,
    IndR_ERA5_sstSON_hypothesis,
) = ca.dim_linregress(tmp, sstHad_SON)

# 后期冬季
tmp = preGPCP_India_JJA_nor[:-1].copy()
tmp.coords["time"] = sstHad_DJF.time
(
    IndR_ERA5_sstnDJF_slope,
    IndR_ERA5_sstnDJF_intercept,
    IndR_ERA5_sstnDJF_rvalue,
    IndR_ERA5_sstnDJF_pvalue,
    IndR_ERA5_sstnDJF_hypothesis,
) = ca.dim_linregress(tmp, sstHad_DJF)


In [ ]:
nIndR_ERA5_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_sstDJF_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_sstDJF_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_sstDJF_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_sstDJF_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_sstDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

nIndR_ERA5_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_sstMAM_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_sstMAM_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_sstMAM_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_sstMAM_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_sstMAM_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

nIndR_ERA5_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_sstJJA_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_sstJJA_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_sstJJA_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_sstJJA_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_sstJJA_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

nIndR_ERA5_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_sstSON_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_sstSON_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_sstSON_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_sstSON_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_sstSON_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

nIndR_ERA5_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_sstnDJF_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_sstnDJF_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_sstnDJF_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_sstnDJF_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

sIndR_ERA5_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_sstDJF_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_sstDJF_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_sstDJF_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_sstDJF_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_sstDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

sIndR_ERA5_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_sstMAM_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_sstMAM_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_sstMAM_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_sstMAM_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_sstMAM_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

sIndR_ERA5_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_sstJJA_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_sstJJA_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_sstJJA_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_sstJJA_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_sstJJA_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

sIndR_ERA5_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_sstSON_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_sstSON_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_sstSON_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_sstSON_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_sstSON_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

sIndR_ERA5_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_sstnDJF_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_sstnDJF_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_sstnDJF_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_sstnDJF_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

IndR_ERA5_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_sstDJF_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_sstDJF_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_sstDJF_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_sstDJF_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_sstDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

IndR_ERA5_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_sstMAM_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_sstMAM_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_sstMAM_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_sstMAM_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_sstMAM_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

IndR_ERA5_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_sstJJA_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_sstJJA_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_sstJJA_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_sstJJA_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_sstJJA_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

IndR_ERA5_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_sstSON_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_sstSON_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_sstSON_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_sstSON_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_sstSON_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

IndR_ERA5_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_sstnDJF_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_sstnDJF_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_sstnDJF_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_sstnDJF_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of HadISST regress onto 1979-2014 "),
)

nIndR_ERA5_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_sstDJF_regress.nc")
nIndR_ERA5_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_sstMAM_regress.nc")
nIndR_ERA5_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_sstJJA_regress.nc")
nIndR_ERA5_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_sstSON_regress.nc")
nIndR_ERA5_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_sstnDJF_regress.nc")

sIndR_ERA5_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_sstDJF_regress.nc")
sIndR_ERA5_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_sstMAM_regress.nc")
sIndR_ERA5_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_sstJJA_regress.nc")
sIndR_ERA5_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_sstSON_regress.nc")
sIndR_ERA5_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_sstnDJF_regress.nc")

IndR_ERA5_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_sstDJF_regress.nc")
IndR_ERA5_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_sstMAM_regress.nc")
IndR_ERA5_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_sstJJA_regress.nc")
IndR_ERA5_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_sstSON_regress.nc")
IndR_ERA5_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_sstnDJF_regress.nc")

### 计算垂直积分水汽通量、水汽通量散度回归

In [ ]:
# 印度半岛北部降水
(
    nIndR_ERA5_uq_slope,
    nIndR_ERA5_uq_intercept,
    nIndR_ERA5_uq_rvalue,
    nIndR_ERA5_uq_pvalue,
    nIndR_ERA5_uq_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, uqERA5_ver_JJA)

(
    nIndR_ERA5_vq_slope,
    nIndR_ERA5_vq_intercept,
    nIndR_ERA5_vq_rvalue,
    nIndR_ERA5_vq_pvalue,
    nIndR_ERA5_vq_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, vqERA5_ver_JJA)

(
    nIndR_ERA5_uqvqdiv_slope,
    nIndR_ERA5_uqvqdiv_intercept,
    nIndR_ERA5_uqvqdiv_rvalue,
    nIndR_ERA5_uqvqdiv_pvalue,
    nIndR_ERA5_uqvqdiv_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, uqvqdivERA5_ver_JJA)

# 印度半岛南部降水
(
    sIndR_ERA5_uq_slope,
    sIndR_ERA5_uq_intercept,
    sIndR_ERA5_uq_rvalue,
    sIndR_ERA5_uq_pvalue,
    sIndR_ERA5_uq_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, uqERA5_ver_JJA)

(
    sIndR_ERA5_vq_slope,
    sIndR_ERA5_vq_intercept,
    sIndR_ERA5_vq_rvalue,
    sIndR_ERA5_vq_pvalue,
    sIndR_ERA5_vq_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, vqERA5_ver_JJA)

(
    sIndR_ERA5_uqvqdiv_slope,
    sIndR_ERA5_uqvqdiv_intercept,
    sIndR_ERA5_uqvqdiv_rvalue,
    sIndR_ERA5_uqvqdiv_pvalue,
    sIndR_ERA5_uqvqdiv_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, uqvqdivERA5_ver_JJA)

# 印度半岛降水
(
    IndR_ERA5_uq_slope,
    IndR_ERA5_uq_intercept,
    IndR_ERA5_uq_rvalue,
    IndR_ERA5_uq_pvalue,
    IndR_ERA5_uq_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, uqERA5_ver_JJA)

(
    IndR_ERA5_vq_slope,
    IndR_ERA5_vq_intercept,
    IndR_ERA5_vq_rvalue,
    IndR_ERA5_vq_pvalue,
    IndR_ERA5_vq_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, vqERA5_ver_JJA)

(
    IndR_ERA5_uqvqdiv_slope,
    IndR_ERA5_uqvqdiv_intercept,
    IndR_ERA5_uqvqdiv_rvalue,
    IndR_ERA5_uqvqdiv_pvalue,
    IndR_ERA5_uqvqdiv_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, uqvqdivERA5_ver_JJA)

In [ ]:
nIndR_ERA5_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_uq_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_uq_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_uq_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_uq_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_uq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_vq_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_vq_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_vq_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_vq_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_vq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ERA5 regress onto 1979-2014 "),
)

nIndR_ERA5_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_ERA5_uqvqdiv_slope.data),
        intercept=(["lat", "lon"], nIndR_ERA5_uqvqdiv_intercept.data),
        rvalue=(["lat", "lon"], nIndR_ERA5_uqvqdiv_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_ERA5_uqvqdiv_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_ERA5_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_uq_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_uq_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_uq_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_uq_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_uq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_vq_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_vq_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_vq_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_vq_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_vq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ERA5 regress onto 1979-2014 "),
)

sIndR_ERA5_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_ERA5_uqvqdiv_slope.data),
        intercept=(["lat", "lon"], sIndR_ERA5_uqvqdiv_intercept.data),
        rvalue=(["lat", "lon"], sIndR_ERA5_uqvqdiv_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_ERA5_uqvqdiv_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_ERA5_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_uq_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_uq_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_uq_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_uq_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_uq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_vq_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_vq_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_vq_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_vq_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_vq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_ERA5_uqvqdiv_slope.data),
        intercept=(["lat", "lon"], IndR_ERA5_uqvqdiv_intercept.data),
        rvalue=(["lat", "lon"], IndR_ERA5_uqvqdiv_rvalue.data),
        pvalue=(["lat", "lon"], IndR_ERA5_uqvqdiv_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_ERA5_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ERA5 regress onto 1979-2014 "),
)
nIndR_ERA5_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_uq_regress.nc")
nIndR_ERA5_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_vq_regress.nc")
nIndR_ERA5_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/non-detrend/nIndR_ERA5_uqvqdiv_regress.nc")

sIndR_ERA5_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_uq_regress.nc")
sIndR_ERA5_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_vq_regress.nc")
sIndR_ERA5_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/non-detrend/sIndR_ERA5_uqvqdiv_regress.nc")

IndR_ERA5_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_uq_regress.nc")
IndR_ERA5_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_vq_regress.nc")
IndR_ERA5_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/non-detrend/IndR_ERA5_uqvqdiv_regress.nc")